<a href="https://colab.research.google.com/github/Legodark/paddok/blob/import_colab/notebooks/machine_learning/PADDOK_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importación de librerias y paquetes

In [1]:
!pip install tensorflow_decision_forests --upgrade > /dev/null

In [88]:
import time

# Tratamiento de lo datos
import numpy as np
import pandas as pd

# Barra de progreso para los bucles
from tqdm.notebook import tqdm

# Visualización
import plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# Machine Learning
# TensorFlow
import tensorflow as tf
from tensorflow import keras
import tensorflow_decision_forests as tfdf

#Skelearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor

%matplotlib inline

Versión de Tensorflow

In [3]:
print(tf.__version__)

2.11.0


Establezco el tema de plotly en modo oscuro

In [4]:
plotly.io.templates.default = "plotly_dark"

### Carga del dataset

In [5]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Curso_IA_BIGDATA/TFM/DataSet/DataSet_Visualizacion/cars_to_visualization.csv'

In [6]:
cars = pd.read_csv(file_path)

Muestro los datos del csv

In [7]:
cars

,Unnamed: 0,localidad,marca,modelo,año,motor (CV),motor (KW),kilometros,combustible,cambio,...,precio,plazas,deposito,0-100 km/h,cilindrada,marchas,marca_id,modelo_id,combustible_id,cambio_id
0,0,Madrid,Citroen,Citroen C3 aircross,2022.0,110.0,81.0,7119.0,Gasolina,Manual,...,19900.0,5.0,45.0,8.8,1199.0,6.0,10.0,118.0,3.0,1.0
1,1,Pontevedra,Renault,Renault Scénic,2017.0,130.0,96.0,43618.0,Gasolina,Manual,...,18975.0,5.0,52.0,12.3,1198.0,6.0,54.0,558.0,3.0,1.0
2,2,Barcelona,Ford,Ford Tourneo courier,2019.0,95.0,71.0,153880.0,Diesel,Manual,...,10900.0,5.0,47.0,0.0,1498.0,5.0,20.0,239.0,1.0,1.0
3,3,Madrid,Renault,Renault Scénic,2019.0,110.0,81.0,73238.0,Diesel,Manual,...,19500.0,5.0,60.0,12.5,1461.0,6.0,54.0,558.0,1.0,1.0
4,4,Pontevedra,Nissan,Nissan Pulsar,2016.0,110.0,81.0,82765.0,Diesel,Manual,...,13500.0,5.0,46.0,11.5,1461.0,6.0,49.0,475.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179497,179497,Barcelona,Mercedes-benz,Mercedes-benz Clase slk,2013.0,184.0,135.0,37379.0,Gasolina,Manual,...,24986.0,2.0,60.0,7.3,1796.0,6.0,44.0,429.0,3.0,1.0
179498,179498,Vizcaya,Mercedes-benz,Mercedes-benz Clase e,2018.0,245.0,180.0,69780.0,Gasolina,Automático,...,39999.0,4.0,66.0,6.6,1991.0,9.0,44.0,415.0,3.0,0.0
179499,179499,Vizcaya,Peugeot,Peugeot 3008,2015.0,121.0,88.0,97945.0,Diesel,Manual,...,13750.0,5.0,60.0,12.0,1560.0,6.0,51.0,512.0,1.0,1.0
179500,179500,Valencia,Mercedes-benz,Mercedes-benz Clase gla,2018.0,136.0,100.0,53193.0,Diesel,Automático,...,29990.0,5.0,50.0,9.1,2143.0,7.0,44.0,418.0,1.0,0.0


Miro las correlaciones de nuevo

In [8]:
cars_correlations = cars.corr()

In [9]:
corr_gen = go.Figure()
corr_gen.add_trace(
    go.Heatmap(
        x = cars_correlations.columns,
        y = cars_correlations.index,
        z = np.array(cars_correlations),
        text=cars_correlations.values,
        texttemplate='%{text:.2f}',
        colorscale=[[0, 'green'], [0.5, 'yellow'], [1, 'red']]
    )
)
corr_gen.show()

Antes de entrar de lleno en el Machine Learning voy a cambiar el nombre a las columnas del DataFrame para no tener problemas de caracteres mas adelante.

In [10]:
# Muestro una linea para ver el nombre de las columna de forma comoda
cars.head(1)

,Unnamed: 0,localidad,marca,modelo,año,motor (CV),motor (KW),kilometros,combustible,cambio,...,precio,plazas,deposito,0-100 km/h,cilindrada,marchas,marca_id,modelo_id,combustible_id,cambio_id
0,0,Madrid,Citroen,Citroen C3 aircross,2022.0,110.0,81.0,7119.0,Gasolina,Manual,...,19900.0,5.0,45.0,8.8,1199.0,6.0,10.0,118.0,3.0,1.0


Cambio el nombre de las columnas

In [11]:
cars = cars.rename(columns={'Unnamed: 0': 'id_car','marca': 'mark', 'modelo': 'model', 'año': 'year', 'motor (CV)': 'horses', 'motor (KW)': 'engine_power', 'kilometros': 'km', 'combustible': 'fuel', 'cambio': 'gearbox', 'emisiones': 'CO2', 'precio': 'price', 'plazas': 'places', 'deposito': 'deposit', '0-100 km/h': '0_100km', 'cilindrada': 'displacement_engine', 'marchas': 'marches'  })

Ahora muestro de nuevo el DataFrame con los nombres cambiados

In [12]:
cars.describe().T

,count,mean,std,min,25%,50%,75%,max
id_car,179502.0,89750.500000,51817.908347,0.0,44875.25,89750.5,134625.75,179501.0
year,179502.0,2016.842046,3.841619,1993.0,2015.00,2018.0,2019.00,2023.0
horses,179502.0,147.338215,74.697602,1.0,110.00,130.0,150.00,887.0
engine_power,179502.0,108.396413,55.132019,9.0,81.00,96.0,110.00,817.0
km,179502.0,81811.256900,56117.663986,0.0,40000.00,74498.0,114000.00,2890000.0
CO2,179502.0,114.994368,48.185070,0.0,103.00,115.0,134.00,495.0
price,179502.0,23926.384798,32034.218607,1.0,13850.00,18990.0,26450.00,1799700.0
places,179502.0,4.965181,0.585251,2.0,5.00,5.0,5.00,9.0
deposit,179502.0,52.278442,14.298763,0.0,45.00,51.0,60.00,121.0
0_100km,179502.0,9.868147,2.725705,0.0,8.60,10.2,11.40,62.0


Ahora lo que realizo es el filtro de datos según las visualizaciones anteriormente vistas, donde determine que lo mejor era quitar los coches con mas de 500.000 kilometros y con un precio superior a 80.000€

Este nuevo DataFrame lo guardare con el nombre de cars_filter

In [13]:
cars_filter = cars[cars['price'] <= 80000]

In [14]:
cars_filter = cars_filter[cars_filter['km'] < 500000]

In [15]:
cars_filter

,id_car,localidad,mark,model,year,horses,engine_power,km,fuel,gearbox,...,price,places,deposit,0_100km,displacement_engine,marches,marca_id,modelo_id,combustible_id,cambio_id
0,0,Madrid,Citroen,Citroen C3 aircross,2022.0,110.0,81.0,7119.0,Gasolina,Manual,...,19900.0,5.0,45.0,8.8,1199.0,6.0,10.0,118.0,3.0,1.0
1,1,Pontevedra,Renault,Renault Scénic,2017.0,130.0,96.0,43618.0,Gasolina,Manual,...,18975.0,5.0,52.0,12.3,1198.0,6.0,54.0,558.0,3.0,1.0
2,2,Barcelona,Ford,Ford Tourneo courier,2019.0,95.0,71.0,153880.0,Diesel,Manual,...,10900.0,5.0,47.0,0.0,1498.0,5.0,20.0,239.0,1.0,1.0
3,3,Madrid,Renault,Renault Scénic,2019.0,110.0,81.0,73238.0,Diesel,Manual,...,19500.0,5.0,60.0,12.5,1461.0,6.0,54.0,558.0,1.0,1.0
4,4,Pontevedra,Nissan,Nissan Pulsar,2016.0,110.0,81.0,82765.0,Diesel,Manual,...,13500.0,5.0,46.0,11.5,1461.0,6.0,49.0,475.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179497,179497,Barcelona,Mercedes-benz,Mercedes-benz Clase slk,2013.0,184.0,135.0,37379.0,Gasolina,Manual,...,24986.0,2.0,60.0,7.3,1796.0,6.0,44.0,429.0,3.0,1.0
179498,179498,Vizcaya,Mercedes-benz,Mercedes-benz Clase e,2018.0,245.0,180.0,69780.0,Gasolina,Automático,...,39999.0,4.0,66.0,6.6,1991.0,9.0,44.0,415.0,3.0,0.0
179499,179499,Vizcaya,Peugeot,Peugeot 3008,2015.0,121.0,88.0,97945.0,Diesel,Manual,...,13750.0,5.0,60.0,12.0,1560.0,6.0,51.0,512.0,1.0,1.0
179500,179500,Valencia,Mercedes-benz,Mercedes-benz Clase gla,2018.0,136.0,100.0,53193.0,Diesel,Automático,...,29990.0,5.0,50.0,9.1,2143.0,7.0,44.0,418.0,1.0,0.0


##### Preparación de los DataFrame para realizar pruebas

DataFrames

In [16]:
# DataFrame sin tratamiento
len(cars)

179502

In [17]:
# DataFrame filtrado por precio y kilometros
len(cars_filter)

176552

In [18]:
cars_not_ids = cars_filter.drop(['id_car', 'marca_id', 'modelo_id', 'combustible_id', 'cambio_id'], axis = 1)

In [19]:
# DataFrame quitando las columnas con id
cars_not_ids

,localidad,mark,model,year,horses,engine_power,km,fuel,gearbox,CO2,price,places,deposit,0_100km,displacement_engine,marches
0,Madrid,Citroen,Citroen C3 aircross,2022.0,110.0,81.0,7119.0,Gasolina,Manual,0.0,19900.0,5.0,45.0,8.8,1199.0,6.0
1,Pontevedra,Renault,Renault Scénic,2017.0,130.0,96.0,43618.0,Gasolina,Manual,129.0,18975.0,5.0,52.0,12.3,1198.0,6.0
2,Barcelona,Ford,Ford Tourneo courier,2019.0,95.0,71.0,153880.0,Diesel,Manual,104.0,10900.0,5.0,47.0,0.0,1498.0,5.0
3,Madrid,Renault,Renault Scénic,2019.0,110.0,81.0,73238.0,Diesel,Manual,105.0,19500.0,5.0,60.0,12.5,1461.0,6.0
4,Pontevedra,Nissan,Nissan Pulsar,2016.0,110.0,81.0,82765.0,Diesel,Manual,94.0,13500.0,5.0,46.0,11.5,1461.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179497,Barcelona,Mercedes-benz,Mercedes-benz Clase slk,2013.0,184.0,135.0,37379.0,Gasolina,Manual,142.0,24986.0,2.0,60.0,7.3,1796.0,6.0
179498,Vizcaya,Mercedes-benz,Mercedes-benz Clase e,2018.0,245.0,180.0,69780.0,Gasolina,Automático,167.0,39999.0,4.0,66.0,6.6,1991.0,9.0
179499,Vizcaya,Peugeot,Peugeot 3008,2015.0,121.0,88.0,97945.0,Diesel,Manual,108.0,13750.0,5.0,60.0,12.0,1560.0,6.0
179500,Valencia,Mercedes-benz,Mercedes-benz Clase gla,2018.0,136.0,100.0,53193.0,Diesel,Automático,108.0,29990.0,5.0,50.0,9.1,2143.0,7.0


In [20]:
cars_not_ep = cars_not_ids.drop(['engine_power'], axis = 1)

In [21]:
# DataFrame quitando la columna de engine_power
cars_not_ep

,localidad,mark,model,year,horses,km,fuel,gearbox,CO2,price,places,deposit,0_100km,displacement_engine,marches
0,Madrid,Citroen,Citroen C3 aircross,2022.0,110.0,7119.0,Gasolina,Manual,0.0,19900.0,5.0,45.0,8.8,1199.0,6.0
1,Pontevedra,Renault,Renault Scénic,2017.0,130.0,43618.0,Gasolina,Manual,129.0,18975.0,5.0,52.0,12.3,1198.0,6.0
2,Barcelona,Ford,Ford Tourneo courier,2019.0,95.0,153880.0,Diesel,Manual,104.0,10900.0,5.0,47.0,0.0,1498.0,5.0
3,Madrid,Renault,Renault Scénic,2019.0,110.0,73238.0,Diesel,Manual,105.0,19500.0,5.0,60.0,12.5,1461.0,6.0
4,Pontevedra,Nissan,Nissan Pulsar,2016.0,110.0,82765.0,Diesel,Manual,94.0,13500.0,5.0,46.0,11.5,1461.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179497,Barcelona,Mercedes-benz,Mercedes-benz Clase slk,2013.0,184.0,37379.0,Gasolina,Manual,142.0,24986.0,2.0,60.0,7.3,1796.0,6.0
179498,Vizcaya,Mercedes-benz,Mercedes-benz Clase e,2018.0,245.0,69780.0,Gasolina,Automático,167.0,39999.0,4.0,66.0,6.6,1991.0,9.0
179499,Vizcaya,Peugeot,Peugeot 3008,2015.0,121.0,97945.0,Diesel,Manual,108.0,13750.0,5.0,60.0,12.0,1560.0,6.0
179500,Valencia,Mercedes-benz,Mercedes-benz Clase gla,2018.0,136.0,53193.0,Diesel,Automático,108.0,29990.0,5.0,50.0,9.1,2143.0,7.0


In [22]:
# DataFrame con marcas que tengas mas de 100 coches usando como base cars_not_ids
cars_up_undred = cars_not_ids['mark'].value_counts() >= 10
cars_up_undred_fl = cars_not_ids[cars_not_ids['mark'].isin(cars_up_undred[cars_up_undred].index)]
cars_up_undred_fl

,localidad,mark,model,year,horses,engine_power,km,fuel,gearbox,CO2,price,places,deposit,0_100km,displacement_engine,marches
0,Madrid,Citroen,Citroen C3 aircross,2022.0,110.0,81.0,7119.0,Gasolina,Manual,0.0,19900.0,5.0,45.0,8.8,1199.0,6.0
1,Pontevedra,Renault,Renault Scénic,2017.0,130.0,96.0,43618.0,Gasolina,Manual,129.0,18975.0,5.0,52.0,12.3,1198.0,6.0
2,Barcelona,Ford,Ford Tourneo courier,2019.0,95.0,71.0,153880.0,Diesel,Manual,104.0,10900.0,5.0,47.0,0.0,1498.0,5.0
3,Madrid,Renault,Renault Scénic,2019.0,110.0,81.0,73238.0,Diesel,Manual,105.0,19500.0,5.0,60.0,12.5,1461.0,6.0
4,Pontevedra,Nissan,Nissan Pulsar,2016.0,110.0,81.0,82765.0,Diesel,Manual,94.0,13500.0,5.0,46.0,11.5,1461.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179497,Barcelona,Mercedes-benz,Mercedes-benz Clase slk,2013.0,184.0,135.0,37379.0,Gasolina,Manual,142.0,24986.0,2.0,60.0,7.3,1796.0,6.0
179498,Vizcaya,Mercedes-benz,Mercedes-benz Clase e,2018.0,245.0,180.0,69780.0,Gasolina,Automático,167.0,39999.0,4.0,66.0,6.6,1991.0,9.0
179499,Vizcaya,Peugeot,Peugeot 3008,2015.0,121.0,88.0,97945.0,Diesel,Manual,108.0,13750.0,5.0,60.0,12.0,1560.0,6.0
179500,Valencia,Mercedes-benz,Mercedes-benz Clase gla,2018.0,136.0,100.0,53193.0,Diesel,Automático,108.0,29990.0,5.0,50.0,9.1,2143.0,7.0


In [23]:
# DataFrame con filtrado de columnas usando como base el DataFrame cars_up_filter
cars_up_filter = cars_up_undred_fl.drop(['engine_power', 'deposit', 'CO2', 'places', '0_100km'], axis = 1)
cars_up_filter

,localidad,mark,model,year,horses,km,fuel,gearbox,price,displacement_engine,marches
0,Madrid,Citroen,Citroen C3 aircross,2022.0,110.0,7119.0,Gasolina,Manual,19900.0,1199.0,6.0
1,Pontevedra,Renault,Renault Scénic,2017.0,130.0,43618.0,Gasolina,Manual,18975.0,1198.0,6.0
2,Barcelona,Ford,Ford Tourneo courier,2019.0,95.0,153880.0,Diesel,Manual,10900.0,1498.0,5.0
3,Madrid,Renault,Renault Scénic,2019.0,110.0,73238.0,Diesel,Manual,19500.0,1461.0,6.0
4,Pontevedra,Nissan,Nissan Pulsar,2016.0,110.0,82765.0,Diesel,Manual,13500.0,1461.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
179497,Barcelona,Mercedes-benz,Mercedes-benz Clase slk,2013.0,184.0,37379.0,Gasolina,Manual,24986.0,1796.0,6.0
179498,Vizcaya,Mercedes-benz,Mercedes-benz Clase e,2018.0,245.0,69780.0,Gasolina,Automático,39999.0,1991.0,9.0
179499,Vizcaya,Peugeot,Peugeot 3008,2015.0,121.0,97945.0,Diesel,Manual,13750.0,1560.0,6.0
179500,Valencia,Mercedes-benz,Mercedes-benz Clase gla,2018.0,136.0,53193.0,Diesel,Automático,29990.0,2143.0,7.0


Selector de DataFrame

In [24]:
# Función para la selección de DataFrame para entrenar prediciones
def select_to_predic(data_frame):
  
  try:
    if data_frame == 1:
      print(f'Has selecionado la opción {data_frame}')
      print(f'Cargando DataFrame...')
      
      dataframe_select = cars
      return dataframe_select
    
    elif data_frame == 2:
      print(f'Has selecionado la opción {data_frame}')
      print(f'Cargando DataFrame...')
      
      dataframe_select = cars_filter
      return dataframe_select
    elif data_frame == 3:
      print(f'Has selecionado la opción {data_frame}')
      print(f'Cargando DataFrame...')
      
      dataframe_select = cars_not_ids
      return dataframe_select
    elif data_frame == 4:
      print(f'Has selecionado la opción {data_frame}')
      print(f'Cargando DataFrame...')
      
      dataframe_select = cars_not_ep
      return dataframe_select
    elif data_frame == 5:
      print(f'Has selecionado la opción {data_frame}')
      print(f'Cargando DataFrame...')
      
      dataframe_select = cars_up_undred_fl
      return dataframe_select
    elif data_frame == 6:
      print(f'Has selecionado la opción {data_frame}')
      print(f'Cargando DataFrame...')
      
      dataframe_select = cars_up_filter
      return dataframe_select
  
  except Exception as e:
    print(f'Se ha producido el siguiente error: {e}')

### Machine Lerning

En principio voy a probar 2 modelos:

1. Random Forest - `tfdf.keras.RandomForestModel`
2. Gradient Boosted Trees Model - `tfdf.keras.GradientBoostedTreesModel`

Por otro lado voy a probar con varios tratamientos de los datos:

1. Todos los datos del CSV
2. Filtrando por precio no seperior a 80.000€ y kilometros no superior a 500.000
3. Filtrando por cantidad de modelos en las marcas, superior a 10.
4. Retirando las columnas de `marca_id`	`modelo_id`	`combustible_id`	`cambio_id`
5. Retirando lo anterior mas alguna columna mas que no tenga mucha correlación con el precio, por ejemplo eliminando `engine_power`
6. Retirando las columnas `engine_power`, `deposit`, `CO2`, `places`, `0_100km` del punto anterior

#### ML con conjunto de datos diferentes

1. Seleciono un DataFrame

In [38]:
sl_dataframe = select_to_predic(int(input('Selecciona un DataFrame: 1 - cars, 2 - cars_filter, 3 - cars_not_ids, 4 - cars_not_ep, 5 - cars_up_undred_fl, 6 - cars_up_filter ')))
sl_dataframe

Selecciona un DataFrame: 1 - cars, 2 - cars_filter, 3 - cars_not_ids, 4 - cars_not_ep, 5 - cars_up_undred_fl, 6 - cars_up_filter 5
Has selecionado la opción 5
Cargando DataFrame...


,localidad,mark,model,year,horses,engine_power,km,fuel,gearbox,CO2,price,places,deposit,0_100km,displacement_engine,marches
0,Madrid,Citroen,Citroen C3 aircross,2022.0,110.0,81.0,7119.0,Gasolina,Manual,0.0,19900.0,5.0,45.0,8.8,1199.0,6.0
1,Pontevedra,Renault,Renault Scénic,2017.0,130.0,96.0,43618.0,Gasolina,Manual,129.0,18975.0,5.0,52.0,12.3,1198.0,6.0
2,Barcelona,Ford,Ford Tourneo courier,2019.0,95.0,71.0,153880.0,Diesel,Manual,104.0,10900.0,5.0,47.0,0.0,1498.0,5.0
3,Madrid,Renault,Renault Scénic,2019.0,110.0,81.0,73238.0,Diesel,Manual,105.0,19500.0,5.0,60.0,12.5,1461.0,6.0
4,Pontevedra,Nissan,Nissan Pulsar,2016.0,110.0,81.0,82765.0,Diesel,Manual,94.0,13500.0,5.0,46.0,11.5,1461.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179497,Barcelona,Mercedes-benz,Mercedes-benz Clase slk,2013.0,184.0,135.0,37379.0,Gasolina,Manual,142.0,24986.0,2.0,60.0,7.3,1796.0,6.0
179498,Vizcaya,Mercedes-benz,Mercedes-benz Clase e,2018.0,245.0,180.0,69780.0,Gasolina,Automático,167.0,39999.0,4.0,66.0,6.6,1991.0,9.0
179499,Vizcaya,Peugeot,Peugeot 3008,2015.0,121.0,88.0,97945.0,Diesel,Manual,108.0,13750.0,5.0,60.0,12.0,1560.0,6.0
179500,Valencia,Mercedes-benz,Mercedes-benz Clase gla,2018.0,136.0,100.0,53193.0,Diesel,Automático,108.0,29990.0,5.0,50.0,9.1,2143.0,7.0


2. Separación del conjunto de datos

In [79]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train, test = train_test_split(sl_dataframe, test_size=0.2, random_state=42)

Muestro conjunto de datos de entrenamiento

In [80]:
train

,localidad,mark,model,year,horses,engine_power,km,fuel,gearbox,CO2,price,places,deposit,0_100km,displacement_engine,marches
153722,Madrid,Ds,Ds Ds3,2016.0,82.0,60.0,88600.0,Gasolina,Manual,107.0,9990.0,5.0,50.0,12.3,1199.0,5.0
19463,Pontevedra,Mercedes-benz,Mercedes-benz Clase gla,2018.0,136.0,100.0,53193.0,Diesel,Automático,108.0,29990.0,5.0,50.0,9.1,2143.0,7.0
42179,Zaragoza,Hyundai,Hyundai I30,2021.0,120.0,88.0,29407.0,Gasolina,Manual,102.0,24750.0,5.0,50.0,11.5,998.0,6.0
171942,Madrid,Mercedes-benz,Mercedes-benz Clase gla,2020.0,122.0,90.0,51745.0,Gasolina,Manual,144.0,28795.0,5.0,50.0,9.0,1595.0,6.0
63996,Sevilla,Bmw,Bmw X2,2018.0,150.0,110.0,47150.0,Diesel,Automático,114.0,29990.0,5.0,51.0,10.0,1995.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121936,Málaga,Audi,Audi A1,2021.0,95.0,70.0,29686.0,Gasolina,Manual,108.0,24900.0,5.0,40.0,10.8,999.0,5.0
105463,Vizcaya,Peugeot,Peugeot 508,2017.0,150.0,110.0,97305.0,Diesel,Manual,110.0,18170.0,5.0,72.0,9.1,1997.0,6.0
134206,Madrid,Audi,Audi A6,2009.0,170.0,125.0,255000.0,Diesel,Automático,153.0,9490.0,5.0,70.0,8.9,1968.0,7.0
149382,Madrid,Kia,Kia Sportage,2021.0,136.0,100.0,24017.0,Diesel,Manual,115.0,28750.0,5.0,0.0,11.2,1598.0,6.0


Muestro conjunto de datos de test

In [81]:
test

,localidad,mark,model,year,horses,engine_power,km,fuel,gearbox,CO2,price,places,deposit,0_100km,displacement_engine,marches
64798,Málaga,Citroen,Citroen Xsara,2000.0,90.0,66.0,250539.0,Diesel,Manual,0.0,1250.0,5.0,54.0,13.2,1997.0,5.0
156568,Vizcaya,Peugeot,Peugeot 308,2014.0,130.0,96.0,68379.0,Gasolina,Manual,104.0,12496.0,5.0,53.0,9.6,1199.0,6.0
127611,Albacete,Ford,Ford Fiesta,2021.0,95.0,70.0,23449.0,Gasolina,Manual,94.0,17940.0,5.0,42.0,10.5,998.0,6.0
1145,Murcia,Volkswagen,Volkswagen Tiguan,2018.0,150.0,110.0,89700.0,Diesel,Automático,126.0,29900.0,5.0,58.0,9.3,1968.0,7.0
46617,Barcelona,Mitsubishi,Mitsubishi Space star,2019.0,80.0,59.0,106200.0,Gasolina,Manual,96.0,11990.0,5.0,35.0,11.7,1193.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81424,Tarragona,Infiniti,Infiniti Fx,2013.0,320.0,235.0,189000.0,Gasolina,Automático,282.0,18990.0,5.0,90.0,6.8,3696.0,7.0
60508,Valencia,Ford,Ford Kuga,2018.0,150.0,110.0,64000.0,Diesel,Manual,164.0,21900.0,5.0,60.0,9.9,1997.0,6.0
172316,Granada,Opel,Opel Crossland,2021.0,110.0,81.0,26893.0,Gasolina,Manual,109.0,17600.0,5.0,45.0,10.9,1199.0,6.0
151321,Alicante,Renault,Renault Kadjar,2016.0,110.0,81.0,154327.0,Diesel,Automático,99.0,15525.0,5.0,55.0,11.7,1461.0,6.0


3. Transformo el DataFrame a DataFrame de TensorFlow


In [82]:
# Especificar la columna de la variable de destino (o target)
target = "price"

# Convertir el DataFrame de pandas a un conjunto de datos de TensorFlow
train_cars = tfdf.keras.pd_dataframe_to_tf_dataset(train, label=target, task=tfdf.keras.Task.REGRESSION)
test_cars = tfdf.keras.pd_dataframe_to_tf_dataset(test, label=target, task=tfdf.keras.Task.REGRESSION)

4. GradientBoostedTreesModel y RandomForestModel

##### A. Construyo el modelo de GradientBoostedTreesModel con 200 arboles y 20 de profundidad

In [83]:
# Especificar las opciones del modelo
model_car_GB = tfdf.keras.GradientBoostedTreesModel(
    task=tfdf.keras.Task.REGRESSION,
    validation_ratio = 0.1,
    num_trees=200,
    max_depth=20)

Use /tmp/tmpdgofs9el as temporary training directory


Entreno el modelo

In [84]:
# Entrenar el modelo
model_car_GB.fit(train_cars)

Reading training dataset...
Training dataset read in 0:00:02.983908. Found 141193 examples.
Training model...
Model trained in 0:01:33.191736
Compiling model...
Model compiled.


Saco un resumen de los que ha realizado el modelo

In [ ]:
# model_car_GB.summary()

Evaluación del modelo con el conjunto de datos de test

In [85]:
# Evaluar el modelo en el conjunto de datos de test
predicted_values = model_car_GB.predict(test_cars)
real_values = test[target].to_numpy()

36/36 [==============================] - 0s 9ms/step


Resultado de las predicciones

In [86]:
predicted_values = predicted_values.flatten()

for i in range(50):
  real_price = real_values[i]
  estim_price = predicted_values[i]
  error_abs = abs(real_price - estim_price)
  print(f"{real_price:6.2f} {estim_price:12.2f} {error_abs:16.2f}")  

1250.00      1973.97           723.97
12496.00     12119.08           376.92
17940.00     16901.07          1038.93
29900.00     28052.00          1848.00
11990.00     11061.83           928.17
11385.00     11003.96           381.04
51490.00     59554.15          8064.15
2990.00      2495.09           494.91
10780.00     10159.15           620.85
16200.00     15933.42           266.58
4200.00      5459.66          1259.66
29300.00     26680.00          2620.00
10300.00     10884.74           584.74
17300.00     15894.75          1405.25
20890.00     21810.82           920.82
21490.00     24074.25          2584.25
23900.00     20658.25          3241.75
20640.00     28825.15          8185.15
10200.00     10415.72           215.72
29900.00     34650.07          4750.07
22650.00     21543.98          1106.02
37999.00     33184.64          4814.36
13500.00     14225.32           725.32
23890.00     25142.92          1252.92
21500.00     21480.25            19.75
13800.00     14400.45       

Coeficiente de determinación

In [87]:
print("Coeficiente de determinación: %.2f" % r2_score(real_values, predicted_values))

Coeficiente de determinación: 0.96


Por último visualizo los arboles de decisión

In [37]:
tfdf.model_plotter.plot_model_in_colab(model_car_GB)

In [38]:
# model_car_GB.make_inspector().variable_importances()

In [39]:
# Accede al historial de entrenamiento
history = model_car_GB.history

##### B. Construyo el modelo de RandomForestModel con 200 arboles y 25 de profundidad

In [40]:
# Especificar las opciones del modelo
model_car_RF= tfdf.keras.RandomForestModel(
    task=tfdf.keras.Task.REGRESSION,
    num_trees=200,
    max_depth=20)

Use /tmp/tmpi0peb7zk as temporary training directory


Entreno el modelo

In [41]:
# Entrenar el modelo
model_car_RF.fit(train_cars)

Reading training dataset...
Training dataset read in 0:00:01.207047. Found 141193 examples.
Training model...
Model trained in 0:05:01.964596
Compiling model...
Model compiled.


Saco un resumen de los que ha realizado el modelo

In [42]:
# model_car_RF.summary()

Evaluación del modelo con el conjunto de datos de test

In [43]:
# Evaluar el modelo en el conjunto de datos de test
predicted_values_RF = model_car_RF.predict(test_cars)
real_values_RF = test[target].to_numpy()

36/36 [==============================] - 10s 243ms/step


Resultado de las predicciones

In [44]:
predicted_values_RF = predicted_values.flatten()

for i in range(20):
  real_price = real_values_RF[i]
  estim_price = predicted_values_RF[i]
  error_abs = abs(real_price - estim_price)
  print(f"{real_price:6.2f} {estim_price:12.2f} {error_abs:16.2f}")   

1250.00      1088.76           161.24
12496.00     12004.99           491.01
17940.00     17358.96           581.04
29900.00     29522.48           377.52
11990.00     10534.58          1455.42
11385.00     10611.68           773.32
51490.00     58343.98          6853.98
2990.00      2606.75           383.25
10780.00     10169.76           610.24
16200.00     16002.88           197.12
4200.00      5642.85          1442.85
29300.00     27922.51          1377.49
10300.00     10796.18           496.18
17300.00     16797.20           502.80
20890.00     21609.32           719.32
21490.00     24542.37          3052.37
23900.00     21541.30          2358.70
20640.00     26089.03          5449.03
10200.00     10742.66           542.66
29900.00     31061.86          1161.86


Coeficiente de determinación

In [45]:
print("Coeficiente de determinación: %.2f" % r2_score(real_values_RF, predicted_values_RF))

Coeficiente de determinación: 0.97


### Predicciones

In [46]:
fig = go.Figure()

fig.update_layout(
    xaxis_title="Precio Predicho",
    yaxis_title="Precio Actual",
    title={
        'text': "Representación gráfica del modelo",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.add_trace(go.Scatter(x=real_values, y=predicted_values, 
                         mode='markers', 
                         marker=dict(size=5, color='#34495E', 
                                     line=dict(width=0.5, color='#196F3D')),
                         name='Precio'))

# Calcular la línea de tendencia utilizando scipy.stats.linregress()
slope, intercept, r_value, p_value, std_err = stats.linregress(real_values, predicted_values)
trendline_x = [min(real_values), max(real_values)]
trendline_y = [slope*xi + intercept for xi in trendline_x]

# Añadir la línea de tendencia a la figura
fig.add_trace(go.Scatter(x=trendline_x, y=trendline_y, 
                         mode='lines', 
                         line=dict(width=2, color='orange'), 
                         name='Tendencia'))


fig.show()

#### Realizando predicción con un coche externo

In [47]:
test[test['mark'] == 'Saab']
test

,localidad,mark,model,year,horses,km,fuel,gearbox,price,displacement_engine,marches
64798,Málaga,Citroen,Citroen Xsara,2000.0,90.0,250539.0,Diesel,Manual,1250.0,1997.0,5.0
156568,Vizcaya,Peugeot,Peugeot 308,2014.0,130.0,68379.0,Gasolina,Manual,12496.0,1199.0,6.0
127611,Albacete,Ford,Ford Fiesta,2021.0,95.0,23449.0,Gasolina,Manual,17940.0,998.0,6.0
1145,Murcia,Volkswagen,Volkswagen Tiguan,2018.0,150.0,89700.0,Diesel,Automático,29900.0,1968.0,7.0
46617,Barcelona,Mitsubishi,Mitsubishi Space star,2019.0,80.0,106200.0,Gasolina,Manual,11990.0,1193.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
81424,Tarragona,Infiniti,Infiniti Fx,2013.0,320.0,189000.0,Gasolina,Automático,18990.0,3696.0,7.0
60508,Valencia,Ford,Ford Kuga,2018.0,150.0,64000.0,Diesel,Manual,21900.0,1997.0,6.0
172316,Granada,Opel,Opel Crossland,2021.0,110.0,26893.0,Gasolina,Manual,17600.0,1199.0,6.0
151321,Alicante,Renault,Renault Kadjar,2016.0,110.0,154327.0,Diesel,Automático,15525.0,1461.0,6.0


In [48]:
# Test
car_predict_full = pd.Series(data={'mark': 'Saab',	'model':	'Saab 9-3', 'year': 2005,	'horses': 150.0, 'km': 190000, 'fuel': 'Diesel',	'gearbox': 'Manual', 'price': 0, 'displacement_engine': 1910.0, 'marches': 6.0})
new_car_df_full = pd.DataFrame([car_predict_full])
car_predict_full = tfdf.keras.pd_dataframe_to_tf_dataset(new_car_df_full, label=target, task=tfdf.keras.Task.REGRESSION)

In [49]:
car_predict_full

<PrefetchDataset element_spec=({'mark': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'model': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'year': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'horses': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'km': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'fuel': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'gearbox': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'displacement_engine': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'marches': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [50]:
# price_car_full = model_car_GB.predict(car_predict_full)
# price_car_full

ValueError: ignored

# Sklearn

In [58]:
cars_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176552 entries, 0 to 179501
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id_car               176552 non-null  int64  
 1   localidad            176552 non-null  object 
 2   mark                 176552 non-null  object 
 3   model                176552 non-null  object 
 4   year                 176552 non-null  float64
 5   horses               176552 non-null  float64
 6   engine_power         176552 non-null  float64
 7   km                   176552 non-null  float64
 8   fuel                 176552 non-null  object 
 9   gearbox              176552 non-null  object 
 10  CO2                  176552 non-null  float64
 11  price                176552 non-null  float64
 12  places               176552 non-null  float64
 13  deposit              176552 non-null  float64
 14  0_100km              176552 non-null  float64
 15  displacement_engi

In [101]:
cars_filter

,id_car,localidad,mark,model,year,horses,engine_power,km,fuel,gearbox,...,price,places,deposit,0_100km,displacement_engine,marches,marca_id,modelo_id,combustible_id,cambio_id
0,0,Madrid,Citroen,Citroen C3 aircross,2022.0,110.0,81.0,7119.0,Gasolina,Manual,...,19900.0,5.0,45.0,8.8,1199.0,6.0,10.0,118.0,3.0,1.0
1,1,Pontevedra,Renault,Renault Scénic,2017.0,130.0,96.0,43618.0,Gasolina,Manual,...,18975.0,5.0,52.0,12.3,1198.0,6.0,54.0,558.0,3.0,1.0
2,2,Barcelona,Ford,Ford Tourneo courier,2019.0,95.0,71.0,153880.0,Diesel,Manual,...,10900.0,5.0,47.0,0.0,1498.0,5.0,20.0,239.0,1.0,1.0
3,3,Madrid,Renault,Renault Scénic,2019.0,110.0,81.0,73238.0,Diesel,Manual,...,19500.0,5.0,60.0,12.5,1461.0,6.0,54.0,558.0,1.0,1.0
4,4,Pontevedra,Nissan,Nissan Pulsar,2016.0,110.0,81.0,82765.0,Diesel,Manual,...,13500.0,5.0,46.0,11.5,1461.0,6.0,49.0,475.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179497,179497,Barcelona,Mercedes-benz,Mercedes-benz Clase slk,2013.0,184.0,135.0,37379.0,Gasolina,Manual,...,24986.0,2.0,60.0,7.3,1796.0,6.0,44.0,429.0,3.0,1.0
179498,179498,Vizcaya,Mercedes-benz,Mercedes-benz Clase e,2018.0,245.0,180.0,69780.0,Gasolina,Automático,...,39999.0,4.0,66.0,6.6,1991.0,9.0,44.0,415.0,3.0,0.0
179499,179499,Vizcaya,Peugeot,Peugeot 3008,2015.0,121.0,88.0,97945.0,Diesel,Manual,...,13750.0,5.0,60.0,12.0,1560.0,6.0,51.0,512.0,1.0,1.0
179500,179500,Valencia,Mercedes-benz,Mercedes-benz Clase gla,2018.0,136.0,100.0,53193.0,Diesel,Automático,...,29990.0,5.0,50.0,9.1,2143.0,7.0,44.0,418.0,1.0,0.0


In [59]:
cars_filter_sk = cars_filter.drop(['id_car', 'localidad', 'mark', 'model', 'fuel', 'gearbox', ], axis = 1)

In [73]:
cars_filter_sk

,year,horses,engine_power,km,CO2,price,places,deposit,0_100km,displacement_engine,marches,marca_id,modelo_id,combustible_id,cambio_id
0,2022.0,110.0,81.0,7119.0,0.0,19900.0,5.0,45.0,8.8,1199.0,6.0,10.0,118.0,3.0,1.0
1,2017.0,130.0,96.0,43618.0,129.0,18975.0,5.0,52.0,12.3,1198.0,6.0,54.0,558.0,3.0,1.0
2,2019.0,95.0,71.0,153880.0,104.0,10900.0,5.0,47.0,0.0,1498.0,5.0,20.0,239.0,1.0,1.0
3,2019.0,110.0,81.0,73238.0,105.0,19500.0,5.0,60.0,12.5,1461.0,6.0,54.0,558.0,1.0,1.0
4,2016.0,110.0,81.0,82765.0,94.0,13500.0,5.0,46.0,11.5,1461.0,6.0,49.0,475.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179497,2013.0,184.0,135.0,37379.0,142.0,24986.0,2.0,60.0,7.3,1796.0,6.0,44.0,429.0,3.0,1.0
179498,2018.0,245.0,180.0,69780.0,167.0,39999.0,4.0,66.0,6.6,1991.0,9.0,44.0,415.0,3.0,0.0
179499,2015.0,121.0,88.0,97945.0,108.0,13750.0,5.0,60.0,12.0,1560.0,6.0,51.0,512.0,1.0,1.0
179500,2018.0,136.0,100.0,53193.0,108.0,29990.0,5.0,50.0,9.1,2143.0,7.0,44.0,418.0,1.0,0.0


In [60]:
X = cars_filter_sk.drop(['price'], axis = 1)

In [61]:
y = cars_filter_sk['price']

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [99]:
rf_model = RandomForestRegressor().fit(X_train.values, y_train.values)
print('Proceso de entrenamiento completado')

Proceso de entrenamiento completado


In [102]:
# Predicción con las entradas de test
y_pred_dtr = rf_model.predict(X_test.values)

In [103]:
y_test_array_dtr = y_test.values

print("Prec. real  Prec. estimado  Error absoluto")
for price in range(20):
  real_price = y_pred_dtr[price]
  estim_price = y_test_array_dtr[price]
  error_abs = abs(real_price - estim_price)
  print(f"{real_price:6.2f} {estim_price:12.2f} {error_abs:16.2f}")

Prec. real  Prec. estimado  Error absoluto
17748.96     17990.00           241.04
19500.00     19500.00             0.00
27566.00     32900.00          5334.00
14990.00     14990.00             0.00
24552.00     24552.00             0.00
15950.00     15950.00             0.00
18900.00     18900.00             0.00
29466.52     26900.00          2566.52
27025.00     27025.00             0.00
16153.47     16200.00            46.53
8937.49      8990.00            52.51
13486.77     13490.00             3.23
13035.00     13035.00             0.00
20125.00     20125.00             0.00
13544.50     13290.00           254.50
11800.00     11800.00             0.00
17595.00     17595.00             0.00
15180.00     15180.00             0.00
21466.36     18900.00          2566.36
20598.92     20990.00           391.08


In [104]:
# Error Cuadrativo
print("Error cuadrático medio: %.2f" % mean_squared_error(y_test_array_dtr, y_pred_dtr, squared=False))

# Coeficiente de determinación: 1 es una predicción perfecta
print("Coeficiente de determinación: %.2f" % r2_score(y_test_array_dtr, y_pred_dtr))

Error cuadrático medio: 1636.43
Coeficiente de determinación: 0.98


In [105]:
fig = go.Figure()

fig.update_layout(
    xaxis_title="Precio Predicho",
    yaxis_title="Precio Actual",
    title={
        'text': "Representación gráfica del modelo",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.add_trace(go.Scatter(x=y_test.values, y=y_pred_dtr, 
                         mode='markers', 
                         marker=dict(size=5, color='#34495E', 
                                     line=dict(width=0.5, color='#196F3D')),
                         name='Precio'))

# Calcular la línea de tendencia utilizando scipy.stats.linregress()
slope, intercept, r_value, p_value, std_err = stats.linregress(y_test.values, y_pred_dtr)
trendline_x = [min(y_test.values), max(y_test.values)]
trendline_y = [slope*xi + intercept for xi in trendline_x]

# Añadir la línea de tendencia a la figura
fig.add_trace(go.Scatter(x=trendline_x, y=trendline_y, 
                         mode='lines', 
                         line=dict(width=2, color='orange'), 
                         name='Tendencia'))


fig.show()

In [71]:
import joblib
joblib.dump(rf_model, 'cars_fit_model.pkl', compress=9)

['cars_fit_model.pkl']